# Data Pre-Processing


## Table of contents

1. [Data Normalization](#normdata)
2. [Deletion of Duplicates](#deldups)
3. [Category Processing](#catproc)
4. [Split the dataset into Train, Val and Test](#split)
4. [Save the model](#save)

Now that we got a feel for the data, let's do some data pre-processing and cleaning before moving towards modeling. 

In this section we will: 

- Process the documents in the dataset with spaCy using the de_core_news_lg pipeline model, so that we can build our baselines and models later on. 

In [94]:
import pandas as pd 
import json
import spacy
from spacy.tokens import Doc
import os
from sklearn.model_selection import train_test_split

In [49]:
# make it deterministic 
import random 
random.seed(42)

Load both pipelines

In [50]:
def load_model(name): 
    nlp = spacy.load(name)
    de_stop_words = nlp.Defaults.stop_words
    return nlp, de_stop_words

In [51]:
def len_doc(doc): 
    doc_length = len(doc)
    return doc_length

def len_doc_only_alpha(doc): 
    doc_length = sum([1 if token.is_alpha else 0 for token in doc])
    return doc_length

In [52]:
def read_dataset(data_path):
    """
    Read a json file and return it as dataframe. 
    Each record should be on a list of json objects inside `data`
    
    args: 
        - data_path, str: path to the dataset to be read
    returns: 
        - Pandas DataFrame with text and label as columns
    """
    with open(data_path, 'r') as f: 
        data = json.loads(f.read())
    
    return pd.json_normalize(data, record_path = ["data"])

In [90]:
dataset_name = "tech_soft_none.json"
data_path = os.path.join(f"./data/raw/{dataset_name}")

In [91]:
skills_df = read_dataset(data_path=data_path)
skills_df.head()

,text,label
0,"Standort Trovarit AG München, Deutschland",none
1,Wir freuen uns auf Ihre Bewerbung unter Angabe...,none
2,Qualifikation zur Heimleitung gemäß Heimperson...,tech
3,Gute organisatorische und konzeptionelle Fähig...,soft
4,"Teamfähigkeit, hohe Flexibilität und Einsatzbe...",soft


In [92]:
large_model = "de_core_news_lg"

nlp_lg, stop_words_lg = load_model(large_model)

In [93]:
# Set the extension on the doc so we can plot it later
Doc.set_extension("doc_len", getter=len_doc, force=True)
Doc.set_extension("doc_len_alpha", getter=len_doc_only_alpha, force=True)

## 1. Data Normalization <a name="normdata"></a>

Dataframe will contain: 
- text: raw text
- label: corresponding to the class
- text_proc: normalized text without punctuation and stop words
- tokens_norm: normalized tokens
- tokens: raw tokens


In [57]:
def normalize(doc): 
    tokens_cleaned = []
    for token in doc: 
        if not token.is_punct and not token.is_stop and not token.is_space:
            tokens_cleaned.append(token.lemma_.lower())
    return tokens_cleaned, " ".join(tokens_cleaned)

In [58]:
%%time
docs = []
tokens = []
tokens_norm = []
text_procs = []
for doc in nlp_lg.pipe(skills_df['text'].to_list(), batch_size=100):
    docs.append(doc)
    tokens.append([token for token in doc])
    tokens_normalized, text_proc = normalize(doc)
    tokens_norm.append(tokens_normalized)
    text_procs.append(text_proc)

CPU times: user 16 s, sys: 335 ms, total: 16.3 s
Wall time: 16.4 s


In [59]:
skills_df['doc'] = pd.Series(docs)    
skills_df['tokens'] = pd.Series(tokens)
skills_df['tokens_norm'] = pd.Series(tokens_norm)
skills_df['text_proc'] = pd.Series(text_procs)
skills_df.head(3)

,text,label,doc,tokens,tokens_norm,text_proc
0,"Standort Trovarit AG München, Deutschland",none,"(Standort, Trovarit, AG, München, ,, Deutschland)","[Standort, Trovarit, AG, München, ,, Deutschland]","[standort, trovarit, münchen, deutschland]",standort trovarit münchen deutschland
1,Wir freuen uns auf Ihre Bewerbung unter Angabe...,none,"(Wir, freuen, uns, auf, Ihre, Bewerbung, unter...","[Wir, freuen, uns, auf, Ihre, Bewerbung, unter...","[freuen, bewerbung, angabe, gehaltsvorstellung...",freuen bewerbung angabe gehaltsvorstellung mög...
2,Qualifikation zur Heimleitung gemäß Heimperson...,tech,"(Qualifikation, zur, Heimleitung, gemäß, Heimp...","[Qualifikation, zur, Heimleitung, gemäß, Heimp...","[qualifikation, heimleitung, gemäß, heimperson...",qualifikation heimleitung gemäß heimpersonalve...


## 2. Delete duplicates for Text column only <a name="deldups"></a>

Delete duplicates from the dataframe, as it might cause issues during training. 

In [65]:
dups_idx = skills_df[skills_df['text'].duplicated() == True].index
dups_idx.shape

(240,)

In [66]:
skills_df_processed = skills_df.drop(index=dups_idx)
skills_df_processed.reset_index(inplace=True, drop=True)

In [67]:
skills_df_processed['text'].nunique()

10009

In [68]:
skills_df_processed.head(3)

,text,label,doc,tokens,tokens_norm,text_proc
0,"Standort Trovarit AG München, Deutschland",none,"(Standort, Trovarit, AG, München, ,, Deutschland)","[Standort, Trovarit, AG, München, ,, Deutschland]","[standort, trovarit, münchen, deutschland]",standort trovarit münchen deutschland
1,Wir freuen uns auf Ihre Bewerbung unter Angabe...,none,"(Wir, freuen, uns, auf, Ihre, Bewerbung, unter...","[Wir, freuen, uns, auf, Ihre, Bewerbung, unter...","[freuen, bewerbung, angabe, gehaltsvorstellung...",freuen bewerbung angabe gehaltsvorstellung mög...
2,Qualifikation zur Heimleitung gemäß Heimperson...,tech,"(Qualifikation, zur, Heimleitung, gemäß, Heimp...","[Qualifikation, zur, Heimleitung, gemäß, Heimp...","[qualifikation, heimleitung, gemäß, heimperson...",qualifikation heimleitung gemäß heimpersonalve...


In [69]:
skills_df_processed['text'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10009 entries, 0 to 10008
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
10009 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


## 3. Category processing <a name="catproc"></a>

We need to process the categories for the model, so that the spaCy models can processed.

In [70]:
def set_categories(label): 
    """
    set_categories will create a dictionary for each label to be passed by assigning True 
    to the label that match and false otherwise.
    
    Example: 
    input: 'tech'
    
    output: {'tech': True, 'soft': False, 'none': False}
    
    args: 
    label: str, representing the category 
    
    return: 
    dict, representing the categories. 
    """
    return dict({'tech': label == 'tech', 
            'soft': label == 'soft',
            'none': label == 'none'
           })

In [71]:
skills_df_processed['cats'] = skills_df_processed['label'].apply(set_categories)
skills_df_processed.head()

,text,label,doc,tokens,tokens_norm,text_proc,cats
0,"Standort Trovarit AG München, Deutschland",none,"(Standort, Trovarit, AG, München, ,, Deutschland)","[Standort, Trovarit, AG, München, ,, Deutschland]","[standort, trovarit, münchen, deutschland]",standort trovarit münchen deutschland,"{'tech': False, 'soft': False, 'none': True}"
1,Wir freuen uns auf Ihre Bewerbung unter Angabe...,none,"(Wir, freuen, uns, auf, Ihre, Bewerbung, unter...","[Wir, freuen, uns, auf, Ihre, Bewerbung, unter...","[freuen, bewerbung, angabe, gehaltsvorstellung...",freuen bewerbung angabe gehaltsvorstellung mög...,"{'tech': False, 'soft': False, 'none': True}"
2,Qualifikation zur Heimleitung gemäß Heimperson...,tech,"(Qualifikation, zur, Heimleitung, gemäß, Heimp...","[Qualifikation, zur, Heimleitung, gemäß, Heimp...","[qualifikation, heimleitung, gemäß, heimperson...",qualifikation heimleitung gemäß heimpersonalve...,"{'tech': True, 'soft': False, 'none': False}"
3,Gute organisatorische und konzeptionelle Fähig...,soft,"(Gute, organisatorische, und, konzeptionelle, ...","[Gute, organisatorische, und, konzeptionelle, ...","[organisatorisch, konzeptionell, fähigkeit]",organisatorisch konzeptionell fähigkeit,"{'tech': False, 'soft': True, 'none': False}"
4,"Teamfähigkeit, hohe Flexibilität und Einsatzbe...",soft,"(Teamfähigkeit, ,, hohe, Flexibilität, und, Ei...","[Teamfähigkeit, ,, hohe, Flexibilität, und, Ei...","[teamfähigkeit, hoch, flexibilität, einsatzber...",teamfähigkeit hoch flexibilität einsatzbereits...,"{'tech': False, 'soft': True, 'none': False}"


## 4. Split dataset into Training, Validation and Testing <a name="split"></a>

Perform data splitting for the dataset with 3 slices with the following proportions: 

- Training: 69% (6975)
- Validation: 15% (1532)
- Testing: 15% (1502)

In [76]:
X_train_raw, X_test, y_train_raw, y_test = train_test_split(skills_df_processed.drop(['label','doc'], axis=1),
                                                    skills_df_processed[['label']], 
                                                    random_state=42, 
                                                    test_size=0.15)

In [77]:
X_train, X_val, y_train, y_val = train_test_split(X_train_raw,
                                                  y_train_raw, 
                                                  random_state=42, 
                                                  test_size=0.18)

In [78]:
X_train.shape[0], X_val.shape[0], X_test.shape[0]

(6975, 1532, 1502)

In [79]:
print(f"Splits \nTraining:{X_train.shape[0]/skills_df_processed.shape[0]*100:.2f} \nDev:{X_val.shape[0]/skills_df_processed.shape[0]*100:.2f}  \nTest:{X_test.shape[0]/skills_df_processed.shape[0]*100:.2f}")

Splits 
Training:69.69 
Dev:15.31  
Test:15.01


## 5. Save the data to build the models <a name="save"></a>

In [80]:
def to_csv(path_to_save, filename, features, label):
    full_path = os.path.join(path_to_save, filename)
    pd.concat([features,label], axis=1).to_csv(full_path, index=False)

In [85]:
!mkdir -p ./data/processed/raw/

In [86]:
to_csv("./data/processed/raw", "training.csv", X_train, y_train)

In [87]:
to_csv("./data/processed/raw", "dev.csv", X_val, y_val)

In [88]:
to_csv("./data/processed/raw", "test.csv", X_test, y_test)

In [89]:
!head -n 3 ./data/processed/test.csv

text,tokens,tokens_norm,text_proc,cats,label
Flexibilität und eigenverantwortliches Handeln zeichnen Sie aus.,"[Flexibilität, und, eigenverantwortliches, Handeln, zeichnen, Sie, aus, .]","['flexibilität', 'eigenverantwortlich', 'handeln', 'zeichnen']",flexibilität eigenverantwortlich handeln zeichnen,"{'tech': False, 'soft': True, 'none': False}",soft
"Durch Ihre Kenntnisse mit Programmier- und Skriptsprachen, stellen Sie sicher, dass die Software den funktionellen wie auch den technischen Spezifikationen genügt.","[Durch, Ihre, Kenntnisse, mit, Programmier-, und, Skriptsprachen, ,, stellen, Sie, sicher, ,, dass, die, Software, den, funktionellen, wie, auch, den, technischen, Spezifikationen, genügt, .]","['kenntnis', 'programmier', 'skriptsprach', 'stellen', 'sicher', 'software', 'funktionell', 'technisch', 'spezifikation', 'genügen']",kenntnis programmier skriptsprach stellen sicher software funktionell technisch spezifikation genügen,"{'tech': True, 'soft': False, 'none': False}",te